In [3]:
%matplotlib inline

In [2]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
import pandas as pd

df = pd.read_csv("rus.txt", sep='\t', header=None).drop(2, axis=1)

In [12]:
df[0] = df[0].apply(lambda x: re.sub(r'[^a-z ]+', '', x.lower()))
df[1] = df[1].apply(lambda x: re.sub(r'[^а-яё ]+', '', x.lower()))
df.sample(10)

,0,1
48613,would tom do that,том бы стал это делать
386402,its difficult to grow anything in this soil,на этой почве трудно чтото вырастить
348196,i knew that tom did that intentionally,я знал что том нарочно это делает
79744,go and see who it is,пойдите посмотрите кто это
272549,tom eats nothing but white meat,том ничего не ест кроме белого мяса
82912,i think youre drunk,помоему ты пьяная
312331,tom dropped his phone in the water,том уронил телефон в воду
179731,i dont even remember that,я этого даже не помню
188799,tom is wearing black shoes,том в чёрных ботинках
211410,youre divorced arent you,ты ведь разведён


In [13]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)

pairs = [p for p in zip(df[0].tolist(), df[1].tolist()) if filterPair(p)]
print(len(pairs), random.choice(pairs))

4650 ('he is very kind to me', 'он очень ко мне добр')


In [14]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [15]:
def prepareData(lang1, lang2, pairs, reverse=False):
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', pairs)
print(random.choice(pairs))

Counted words:
eng 2190
rus 4163
('she shut the door', 'она закрыла дверь')


The Encoder
-----------





In [58]:
class EncoderRNN(nn.Module):
    def __init__(self, rnn_type, input_size, hidden_size, num_layers=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = rnn_type(hidden_size, hidden_size, num_layers=num_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        embedded = embedded.view(1, 1, -1)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device),
                    torch.zeros(self.num_layers, 1, self.hidden_size, device=device))
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

The Decoder
-----------




In [59]:
class DecoderRNN(nn.Module):
    def __init__(self, rnn_type, hidden_size, output_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = rnn_type(hidden_size, hidden_size, num_layers=num_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(1, 1, self.hidden_size, device=device),
                    torch.zeros(1, 1, self.hidden_size, device=device))
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [19]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [20]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [21]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [22]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [23]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [24]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# GRU с одним рекуррентным слоем



In [55]:
hidden_size = 256
encoder1 = EncoderRNN(nn.GRU, input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(nn.GRU, hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 41s (- 9m 42s) (5000 6%) 4.1467
1m 20s (- 8m 44s) (10000 13%) 3.6465
2m 0s (- 8m 0s) (15000 20%) 3.0068
2m 40s (- 7m 21s) (20000 26%) 2.4851
3m 21s (- 6m 42s) (25000 33%) 2.0751
4m 2s (- 6m 3s) (30000 40%) 1.6416
4m 43s (- 5m 24s) (35000 46%) 1.3666
5m 26s (- 4m 45s) (40000 53%) 1.1402
6m 7s (- 4m 4s) (45000 60%) 0.9248
6m 49s (- 3m 24s) (50000 66%) 0.7586
7m 30s (- 2m 43s) (55000 73%) 0.6333
8m 11s (- 2m 2s) (60000 80%) 0.5680
8m 53s (- 1m 22s) (65000 86%) 0.5042
9m 35s (- 0m 41s) (70000 93%) 0.4733
10m 17s (- 0m 0s) (75000 100%) 0.4309


In [56]:
evaluateRandomly(encoder1, decoder1)

> he is one of my neighbours
= он один из моих соседей
< он один из моих соседей <EOS>

> she is washing the car
= она моет машину
< она моет машину <EOS>

> i am the same age
= я того же возраста
< мне столько же лет <EOS>

> i am sick
= мне дурно
< я болен <EOS>

> they are on good terms with their neighbors
= у них хорошие отношения с соседями
< они в хороших отношениях с соседями <EOS>

> i am grateful to them
= я благодарен им
< я им признателен <EOS>

> they are not coming today
= они сегодня не придут
< они сегодня не придут <EOS>

> we are worried about you
= мы за тебя беспокоимся
< мы за вас беспокоимся <EOS>

> she is eager to live in australia
= она стремится жить в австралии
< она стремится жить в австралии <EOS>

> you are always as busy as a bee
= ты всегда трудишься как пчела
< ты всегда трудишься как пчела <EOS>



# GRU с двумя рекуррентными слоями

In [50]:
hidden_size = 256
encoder2 = EncoderRNN(nn.GRU, input_lang.n_words, hidden_size, 2).to(device)
decoder2 = DecoderRNN(nn.GRU, hidden_size, output_lang.n_words, 2).to(device)

trainIters(encoder2, decoder2, 75000, print_every=5000)

0m 51s (- 12m 4s) (5000 6%) 4.1548
1m 41s (- 10m 56s) (10000 13%) 3.8152
2m 31s (- 10m 6s) (15000 20%) 3.2783
3m 22s (- 9m 16s) (20000 26%) 2.7649
4m 13s (- 8m 27s) (25000 33%) 2.3014
5m 4s (- 7m 37s) (30000 40%) 1.9008
5m 57s (- 6m 48s) (35000 46%) 1.5721
6m 49s (- 5m 58s) (40000 53%) 1.2825
7m 41s (- 5m 7s) (45000 60%) 1.0686
8m 34s (- 4m 17s) (50000 66%) 0.8893
9m 26s (- 3m 26s) (55000 73%) 0.7429
10m 18s (- 2m 34s) (60000 80%) 0.6584
11m 11s (- 1m 43s) (65000 86%) 0.5767
12m 3s (- 0m 51s) (70000 93%) 0.5253
12m 56s (- 0m 0s) (75000 100%) 0.4645


In [51]:
evaluateRandomly(encoder2, decoder2)

> i am familiar with this neighborhood
= мне знаком этот район
< мне знаком этот район <EOS>

> she is kissing him
= она его целует
< она целует его <EOS>

> she is a secondrate singer at best
= она в лучшем случае второсортная певичка
< она в лучшем случае второсортная певица <EOS>

> she is appearing on tv tonight
= вечером её будут показывать по телевизору
< вечером её будут показывать по телевизору <EOS>

> you are going to have to pay for it
= вам придётся за него заплатить
< тебе придётся за неё заплатить <EOS>

> he is about forty
= ему лет сорок
< ему около сорока <EOS>

> she still loved him
= она всё ещё любила его
< она всё ещё любила его <EOS>

> you arent canadian
= ты не канадка
< вы не канадка <EOS>

> i am not leaving you
= я вас не покидаю
< я вас не покидаю <EOS>

> i am eating an apple
= я ем яблоко
< я ем яблоко <EOS>



# LSTM с одним рекуррентным слоем

In [62]:
hidden_size = 256
encoder3 = EncoderRNN(nn.LSTM, input_lang.n_words, hidden_size).to(device)
decoder3 = DecoderRNN(nn.LSTM, hidden_size, output_lang.n_words).to(device)

trainIters(encoder3, decoder3, 75000, print_every=5000)

0m 42s (- 9m 53s) (5000 6%) 4.3116
1m 22s (- 8m 54s) (10000 13%) 3.9452
2m 2s (- 8m 10s) (15000 20%) 3.3706
2m 43s (- 7m 29s) (20000 26%) 2.8550
3m 25s (- 6m 50s) (25000 33%) 2.3989
4m 7s (- 6m 10s) (30000 40%) 1.9761
4m 49s (- 5m 30s) (35000 46%) 1.6374
5m 31s (- 4m 49s) (40000 53%) 1.3771
6m 13s (- 4m 8s) (45000 60%) 1.1575
6m 54s (- 3m 27s) (50000 66%) 0.9662
7m 36s (- 2m 46s) (55000 73%) 0.8082
8m 18s (- 2m 4s) (60000 80%) 0.7095
9m 1s (- 1m 23s) (65000 86%) 0.6109
9m 43s (- 0m 41s) (70000 93%) 0.5577
10m 25s (- 0m 0s) (75000 100%) 0.4969


In [63]:
evaluateRandomly(encoder3, decoder3)

> he isnt here because hes ill
= его нет потому что он болеет
< его нет нет потому что он <EOS>

> she is a secondrate singer at best
= она в лучшем случае второсортная певичка
< она в лучшем случае второсортная певица <EOS>

> he is quite a gentleman
= он действительно джентльмен
< он действительно джентльмен <EOS>

> he is pleased with his new car
= он доволен своим новым автомобилем
< он доволен своей новой машиной <EOS>

> she is younger than me
= она младше меня
< она младше меня <EOS>

> i am wasting my time
= я зря трачу время
< я зря трачу время <EOS>

> he is in the bathroom
= он в ванной
< он в ванной <EOS>

> i am a tennis player
= я теннисистка
< я теннисистка <EOS>

> they are in class
= они в классе
< они в классе <EOS>

> he is not a doctor but a teacher
= он не врач он учитель
< он не доктор а доктор <EOS>



# LSTM с двумя рекуррентными слоями

In [60]:
hidden_size = 256
encoder3 = EncoderRNN(nn.LSTM, input_lang.n_words, hidden_size, 2).to(device)
decoder3 = DecoderRNN(nn.LSTM, hidden_size, output_lang.n_words, 2).to(device)

trainIters(encoder3, decoder3, 75000, print_every=5000)

0m 52s (- 12m 20s) (5000 6%) 4.3639
1m 43s (- 11m 14s) (10000 13%) 4.1525
2m 35s (- 10m 23s) (15000 20%) 3.9034
3m 28s (- 9m 32s) (20000 26%) 3.5555
4m 20s (- 8m 41s) (25000 33%) 3.1415
5m 14s (- 7m 51s) (30000 40%) 2.7507
6m 7s (- 6m 59s) (35000 46%) 2.3641
7m 0s (- 6m 8s) (40000 53%) 1.9821
7m 55s (- 5m 16s) (45000 60%) 1.7031
8m 48s (- 4m 24s) (50000 66%) 1.4201
9m 42s (- 3m 31s) (55000 73%) 1.2199
10m 37s (- 2m 39s) (60000 80%) 1.0632
11m 31s (- 1m 46s) (65000 86%) 0.8923
12m 25s (- 0m 53s) (70000 93%) 0.7687
13m 19s (- 0m 0s) (75000 100%) 0.6553


In [61]:
evaluateRandomly(encoder3, decoder3)

> you arent in a hurry are you
= ты же не торопишься
< вы же не торопишься <EOS>

> he is employed on the farm
= он нанялся на ферму
< его наняли на ферму <EOS>

> he is an unsung hero
= он  невоспетый герой
< он  невоспетый герой <EOS>

> we are all eager to know the truth
= мы все жаждем знать правду
< мы все жаждем знать правду <EOS>

> we arent going to sing that song
= мы не будем петь эту песню
< мы не будем петь эту песню <EOS>

> you are always watching tv
= ты всё время смотришь телевизор
< вы всё время смотрите телевизор <EOS>

> he is capable of doing it
= он способен сделать это
< он способен сделать это <EOS>

> i am downloading books
= я скачиваю книги
< я скачиваю книги <EOS>

> he is a biologist
= он биолог
< он биолог <EOS>

> i am counting on you
= я на вас рассчитываю
< я на вас рассчитываю <EOS>

